# Bias field correction

Attempt 1 : use sitk

### import libraries

In [ ]:
import os       # using operating system dependent functionality (folders)
import glob
import pandas as pd # data analysis and manipulation
import numpy as np    # numerical computing (manipulating and performing operations on arrays of data)
import copy     # Can Copy and Deepcopy files so original file is untouched.
from ipywidgets import IntSlider, Output
import ipywidgets as widgets
from IPython.display import display
import matplotlib.pyplot as plt
import SimpleITK as sitk
import skimage
import hashlib
import sys
sys.path.insert(0, '../') # path to functions
from brainspin import file_handler as fh # 
from brainspin import mold #
from brainspin import carve
from brainspin.file_handler import Config

In [ ]:
### Let's see how it works on a 2D, then on a nifti 

In [ ]:
config = Config()
root_mri_directory = config.get_directory('root_mri_directory')
mri_pattern = os.path.join(root_mri_directory, '**/*.gz')
gz_files = glob.glob(mri_pattern, recursive=True)

In [ ]:

def show_me_debiased(image_filename):
    # reading in on sitk.sitkUInt8 may be wrong, but anyways
    inputImage = sitk.ReadImage(image_filename,sitk.sitkUInt8)

    maskImage = sitk.OtsuThreshold( inputImage, 0, 1, 200 )

    inputImage = sitk.Cast( inputImage, sitk.sitkFloat32 )

    corrector = sitk.N4BiasFieldCorrectionImageFilter();

    numberFittingLevels = 4

    output = corrector.Execute( inputImage, maskImage )

    outputCasted = sitk.Cast(output,sitk.sitkVectorUInt8)
    
    return outputCasted

In [ ]:
outputCasted = show_me_debiased('test/N4BiasFieldCorrection_input_image.png')
inputImage = sitk.ReadImage('test/N4BiasFieldCorrection_input_image.png',sitk.sitkUInt8)

In [ ]:
npa = sitk.GetArrayViewFromImage(outputCasted)
opa = sitk.GetArrayViewFromImage(inputImage)
plt.subplot(1, 2, 1) # row 1, col 2 index 1
plt.imshow(npa,cmap='Greys',  interpolation='nearest')
plt.title("Output image")
plt.xlabel('X-axis ')
plt.ylabel('Y-axis ')

plt.subplot(1, 2, 2) # index 2
plt.imshow(opa,cmap='Greys',  interpolation='nearest')
plt.title("Original image")
plt.xlabel('X-axis ')
plt.ylabel('Y-axis ')

plt.show()

In [ ]:
outputCasted = show_me_debiased(gz_files[0])


# Read the .nii image containing the volume with SimpleITK:
t1_fn = gz_files[0]
sitk_t1 = sitk.ReadImage(t1_fn)

# and access the numpy array:
t1 = sitk.GetArrayFromImage(sitk_t1)

In [ ]:
mold.SliceViewer(t1)

In [ ]:
#sitk.GetArrayFromImage?

In [ ]:
output = sitk.GetArrayFromImage(outputCasted)

In [ ]:
mold.SliceViewer(output)